In [0]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../src")))

from dfutility_helper.utils import add_map
from pyspark.sql.functions import timestamp_diff, to_date, col, expr, lit, to_timestamp_ntz, trim

In [0]:
params = dbutils.widgets.getAll()
catalog = params.pop('catalog')

In [0]:
city_bike_df = spark.read \
    .table(f"{catalog}.`01-bronze`.jc_city")

In [0]:
city_bike_df = city_bike_df.withColumns({
    "started_at": to_timestamp_ntz(trim(col("started_at"))),
    "ended_at": to_timestamp_ntz(trim(col("started_at"))),
    "trip_start_date": to_date(col("started_at")),
    "trip_duration_mins": timestamp_diff("MINUTE", col("started_at"), col("ended_at"))
}) \
    .selectExpr(
        "ride_id",
        "trip_start_date",
        "started_at",
        "ended_at",
        "start_station_name",
        "end_station_name",
        "trip_duration_mins"
    )

city_bike_df = add_map(city_bike_df, **params)

In [0]:
city_bike_df.write \
    .format("delta") \
        .mode("overwrite") \
            .saveAsTable(f"{catalog}.`02-silver`.city_bike_enriched")